In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from astropy.constants import c, k_B, M_sun, G, m_p, sigma_sb, m_p
import astropy.units as u
import numpy as np
import numba
import sys

sys.path.append('../Numerical_methods')
from matrix_calculator import A_matrix
from scheme_calculator import forward_backward
from bachelor_funcs import get_1_dev_new, Σ_initial, r_array

# Set som plotting standards:
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 12}
mpl.rc('font', **font)

### Equation 10 from Suzuki

$$  \frac{\partial\Sigma}{\partial t} - \frac{1}{r}\frac{\partial}{\partial r}\left[
    \frac{2}{r\Omega}\left\{
    \frac{\partial}{\partial r}\left(
    r^2\Sigma\alpha_{r\phi}c_s^2
    \right)
    + r^2\alpha_{\phi z}(\rho c_s^2)_{\textrm{mid}}
    \right\}\right]
    + C_{\textrm{w}}(\rho c_s)_{\textrm{mid}}
    \;=\;0$$

$$
\Omega\approx\Omega_K=\sqrt{\frac{GM_*}{r^3}}
$$    

$$
c_s^2=\frac{k_BT}{\mu m_H}
$$

#### Neglecting disc wind and torque:
$$  \frac{\partial\Sigma}{\partial t} - \frac{1}{r}\frac{\partial}{\partial r}\left[
    \frac{2}{r\Omega}\left\{
    \frac{\partial}{\partial r}\left(
    r^2\Sigma\alpha_{r\phi}c_s^2
    \right)
    \right\}\right]
    
    \;=\;0$$

#### From B.10:
$$\sigma_\textrm{SB}T^4=\frac{3}{4}\Omega\Sigma\bar{\alpha_{r\phi}}c_s^2$$

$$\Rightarrow \quad c_s^6\;=\;\frac{1}{\sigma_\textrm{SB}}
\left(\frac{\mu m_{H}}{K_B}\right)^{-4}
\frac{3}{4}\Omega\Sigma \bar{\alpha_{r\phi}}
$$

In [10]:
μ = 2.34    #(Hayashi 1981)

In [36]:
def Omega_kep(r):
    return np.sqrt((G * M_sun) / r**3)


def c_s(Ω,  Σ, α_rφ = 8e-3):
    return ((sigma_sb**(-1) * (μ * m_p / k_B)**(-4) * 3/4 * Ω * Σ * α_rφ)**(1/6)).to('m/s')

In [39]:
r = r_array * u.au
c_s(Omega_kep(r), Σ_initial(r))[0]

<Quantity 9063.54972517 m / s>

### Imposing the boundary condition $\frac{\partial}{\partial r}(\Sigma r^{3/2})=0$

$$\frac{\partial}{\partial r}(\Sigma r^{3/2})=0$$
$$\Rightarrow\frac{\partial}{\partial r}(\Sigma)\cdot r^{3/2}\;+\;\frac{3}{2}\Sigma r^{1/2}=0$$
$$\Sigma r^{2}=-\frac{2}{3}\;r^{3}\;\frac{\partial}{\partial r}\Sigma$$


In [ ]:
def one_alpha_diff(t, Σ):
    boundary = -2/3 * x**(3) * get_first_dev(Σ, Δx)

    inner = Σ * x**(1/2)
    inner[0] = boundary[0]
    inner[-1] = boundary[-1]

    inner_dev = get_first_dev(inner, Δx)
    
    outer_dev = get_first_dev(x**(1/2) * inner_dev, Δx)

    #outer_dev[0] = 0
    #outer_dev[-1] = 0

    return 3/x * outer_dev